## 讀取含有設定字元的標題之信件並下載附件

In [1]:
# %%time    # ipython語法，將會給出cell的代碼運行一次所花費的時間。
import configparser
import easygui as g
import win32com.client
from pathlib import Path
from datetime import date, datetime, timedelta
from sys import exit

# 把預設檔內的資料編輯成字典
cfg = configparser.ConfigParser()
cfg.read(r'outlook.ini', encoding = 'utf_8_sig')
cfg = cfg['DEFAULT']
cfg_dict = {i: (cfg.get(i)) for i in cfg}

# 給預設的防呆
if len(cfg_dict) < 5:
    readfolder = 'Inbox'   # 抓取郵件資料夾，預設為Inbox
    subject = None         # 預設標題包含的字串
    startdate = 14         # 抓多少天的信
    savefolderpath = 'D:\Outlook' # 信中附件要存的地方
    logfilename = 'D:\log.txt'    # 使用紀錄檔案

# 把 cfg 檔中的key都直接轉變數，其值為val
for key, val in cfg_dict.items(): 
    locals()[key] = val

Path(savefolderpath).mkdir(parents = True, exist_ok = True) # 目錄不存在的話，就建立(parents=True允許父目錄不存在)
sdt = (date.today() - timedelta(days = int(startdate))).strftime('%Y/%m/%d') # 多少天前，start date
msg = '郵件篩選規則'
title = '請輸入'
fields =  ['郵件資料夾 (預設)','郵件標題包含字元','收件時間 >= (年/月/日)','檔案儲存路徑']
values = [readfolder, subject, sdt, savefolderpath]
fv = g.multenterbox(msg, title, fields, values)     # 跳出 多輸入框 (按下取消回傳None，按下確定回傳包含內部所有數值的list)

while True:
    if fv is None:
        g.msgbox(msg = '作業取消')   # 跳出 message box
        exit(0)
    errmsg = ''
    for i in range(len(fv)):        # 檢查輸入是否有漏
        if fv[i].strip() == '':     # 若字串去除前後空格後為空值，回傳該欄位為必要之訊息
            errmsg += ('"%s" 是必要欄位.\n\n' % fields[i])  
    if errmsg == '':                # 代表檢查無誤 直接跳出while迴圈進入下一步
        break
    fv = g.multenterbox(errmsg, title, fields, values)     # 否則就要再key資料

# 跳出多選欄位，選擇後按下確定會回傳所選項目的list
ft = g.multchoicebox(
    msg = '附件檔案類型', 
    title = '至少選擇一個', 
    choices = ['pdf','xls','ppt','doc','msg','txt','csv','zip','7z','rar']
)
if ft is None:
    g.msgbox(msg = '作業取消')
    exit(0)


outlook = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')

# for i in range(90):     # 可以看到GetDefaultFolder中不同數字對應不同的區塊(但非每個數字都有對應)
#     try: print('{}     {}'.format(i, outlook.GetDefaultFolder(i)))
#     except: continue
        
if fv[0].lower() == 'inbox':  # 確定郵件資料夾名稱
    inbox = outlook.GetDefaultFolder(6)
else:
    inbox = outlook.GetDefaultFolder(6).Folders[fv[0]]   # 收件夾內部的資料夾

# Filter = ("@SQL=" + chr(34) + "urn:schemas:httpmail:subject" + chr(34) + " Like '%RPA%' AND " + chr(34) + "urn:schemas:httpmail:datereceived" + chr(34) + "> '2021/02/01' AND" + chr(34) + "urn:schemas:httpmail:hasattachment" + chr(34) + "=1")

# 此為DASL查詢語法
Filter = ('@SQL=' + chr(34) + "urn:schemas:httpmail:subject" +      # chr(34) 是 " 這個符號
                    chr(34) + " Like '%" + fv[1] + "%' AND " + 
                    chr(34) + "urn:schemas:httpmail:datereceived" + 
                    chr(34) + "> '" + fv[2] + "' AND" +
                    chr(34) + "urn:schemas:httpmail:hasattachment" + 
                    chr(34) + "= 1")

items = inbox.Items#.Restrict(Filter) # 篩選郵件
# items.Sort('[ReceivedTime]', True) # 依照收件日期由新到舊排序

with open(logfilename, 'a+', encoding = 'utf_8_sig') as log:  # 用a+是將資料讀寫在舊檔案之後
    print(datetime.now().strftime('%Y/%m/%d %H:%M 匯出紀錄:'), file = log)  # print 在 log 中
    for i in items: # 掃每一封信
        # SenderName: 寄件者
        # Subject: 主旨
        # SentOn: 寄件時間
        # Attachments: 檢查內部所有的附件(包含圖片)，如果要附件"名稱"，需要在後面加".FileName"
        print('|'.join([i.SenderName, i.Subject, i.SentOn.strftime('%m/%d %H:%M')]), file = log) # join為將list內的元素轉成str並用前面的符號隔開
        try:
            for j in i.Attachments:
                if j.FileName.rsplit('.', 1)[-1][:3] in ft: # 附檔格式符合才存檔
                    print('--> 附檔:', j.FileName, file = log)
                    j.SaveAsFile(fv[3] + '\\' + j.FileName)  # fv[3] 為路徑
                else:
                    continue
        except:
            print('--> Skip, 檔案無法解析', j.DisplayName, file = log)
            pass

with open(logfilename, encoding = 'utf_8_sig') as f:
    l_msg = f.readlines()       # 讀取log裡面的全部內容

lix = l_msg.index(''.join([i for i in l_msg if ' 匯出紀錄:' in i][-1]))
lmsg = l_msg[lix:]
g.textbox(msg = '匯出紀錄', title = '此次使用匯出紀錄', text = ''.join(lmsg))   # 跳出包含log內容的text box

'2021/06/25 17:06 匯出紀錄:\nJudy Sung [宋佩眞]|歡迎加入泰詠的行列！|05/18 10:11\n--> 附檔: 新人導引指南(110.4.19).pdf\nLydia Chen [陳麗君]|（麗君OK，無內文） RE: HR目前因應疫情，建備完整的連絡資訊，請各位確認自己的資訊是否正確，謝謝~|05/18 08:54\nLisa Lin [林逸嬅]|RE: HR目前因應疫情，建備完整的連絡資訊，請各位確認自己的資訊是否正確，謝謝~|05/18 08:47\nMinter Hsieh [謝敏塘]|SAP 資料|05/19 11:29\nVicky Chou [周美津]|（股東會的防疫計劃）公司股東會即將於5月27日(四)召開，請各位一起共襄盛舉，請協助以下項目，謝謝。|05/20 11:03\nChen KJ|測試信|05/20 11:36\nJoanie Yu [余翠玲]|RE: QC使用問題反饋-增加DIP/組測隨線QC問題|05/21 15:10\nSean Cheng [鄭志雄]|RE: QC使用問題反饋-增加DIP/組測隨線QC問題|05/21 15:49\nJoanie Yu [余翠玲]|RE: QC使用問題反饋-增加DIP/組測隨線QC問題|05/21 16:07\nPapa Chiu [邱思慧]|RE: MES成品出貨無法使用|05/25 14:34\nSean Cheng [鄭志雄]|RE: MES成品出貨無法使用|05/25 14:25\nPapa Chiu [邱思慧]|MES成品出貨無法使用|05/25 13:59\nVicky Chou [周美津]|5/26（三）早上9點【資訊部—相關進行項目之議題討論】若此時間有任何不妥，請再告訴我，謝謝~|05/26 08:11\nJoan Wu [吳佳謹]|RE: QC使用問題反饋-增加DIP/組測隨線QC問題|05/26 13:21\nPapa Chiu [邱思慧]|RE: MES成品出貨無法使用|05/26 14:01\nTina Lin [林月美]|   麻煩請幫我安裝  SAP|05/28 16:38\nMinter Hsieh [謝敏塘]|RE: python安裝SOP|05/28 16:38\nFang Shen [沈淑芳]|RE: 退料系統異常|05/30 

## 寄信

In [14]:
import win32com.client
import warnings
import sys
import pythoncom

warnings.filterwarnings('ignore')
pythoncom.CoInitialize()

sub = '用python幫outlook寄信測試'
body = '主旨'
receivers = ['kj_chen@topunion.com.tw;onion0v0jack@gmail.com']   # 多人用;隔開  wendy_hung@topunion.com.tw
atta_list = [r'D://Topunion_KJ//Python_related//outlook_file_downloader//testfile.txt']


def sendmail(subject, body, receivers, atta_list):
    outlook = win32com.client.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = receivers[0]
    mail.Subject = subject#.decode('utf-8')    # 主旨
    mail.Body = body#.decode('utf-8')      # 內文
    for atta in atta_list:
        mail.Attachments.Add(atta)   # 附件
    mail.Send()

sendmail(subject = sub, body = body, receivers = receivers, atta_list = atta_list)


In [18]:
164917510/547383692

0.3012831993540648